
Feedback to shins777@gmail.com

이 Colab 은 다양한 비즈니스 요구사항에 적용할수 있는 이미지 분석에 대한 예제입니다.
Gemini Pro vision을 활용해서 이미지를 분석합니다.

### 참고: 이곳에서 사용되는 이미지는 교육목적으로 웹상에서 다운로드 받은 이미지입니다. 상업적인 용도로 사용하지 말아주세요.
이미지에 대한 저작권 문제가 있다면 해당 이미지를 삭제하도록 하겠습니다.

# Library 설치

In [3]:
!pip install --upgrade --quiet google-cloud-aiplatform

# GCP 사용자 인증

In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
model_name="gemini-pro-vision"
project="ai-hangsik"
location="asia-northeast3"

In [6]:
from google.cloud import aiplatform
aiplatform.init(project=project)

# Gemini Pro vision - Multimodal

이미지 인코딩을 위한 함수

In [7]:
import base64
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part


def get_obj_content(coding='utf-8', obj_loc=""):
    with open(obj_loc, 'rb') as f:
        obj_data = base64.b64encode(f.read()).decode(coding)
        return obj_data


# Image 분석

예제에서 제시된 이미지는 Gemini Pro Vision 테스트용 이외의 목적으로 상업적으로 사용하지 말아주세요.

##주식 챠트 이미지 분석

In [8]:


def generate_image(file_path, prompt):

  img_data = get_obj_content(obj_loc=file_path)
  image_obj = Part.from_data(data=base64.b64decode(img_data), mime_type="image/png")

  model = GenerativeModel(model_name)
  responses = model.generate_content(
                                    [image_obj, prompt],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings=[],
    stream=False,
  )

  return responses.text

#------

file_path = "./Kospi.png"

prompt = """이미지에 대해서 아래 내용을 중심으로 자세하게 설명해주세요.
1. 몇일날짜의 정보입니다.
2. 52주 최고가는 얼마인가요 ?
3. 그래프의 변동 상황을 시간별로 정리해주세요.
4. 그래프가 생성된 다음날의 상황은 어떻게 예측하나요 ? 이유도 알려주세요.
5. 이 그래프는 일반기업의 그래프인가요 ?
6. 이 그래프는 어떤 데이터를 기반으로 만들어졌나요 ?

"""

result = generate_image(file_path, prompt)
print(result)



 1. 2023년 3월 7일 화요일 오후 6시 5분 기준 정보입니다.
2. 52주 최고가는 2,694.80입니다.
3. 오전 9시 30분에 2,653.98로 시작해서, 오후 12시경까지 2,640까지 하락했습니다. 그 후 오후 3시까지 2,650까지 상승했습니다.
4. 다음날 상황은 오르락 내리락 할 것으로 예상합니다. 전날보다 약간 오를 수도 있고, 내릴 수도 있습니다. 이는 지난 5일간의 그래프가 오르락 내리락하는 상황이었기 때문입니다.
5. 이 그래프는 한국거래소의 종합주가지수인 KOSPI의 그래프입니다.
6. 이 그래프는 한국거래소에서 거래되는 모든 주식의 가격을 기반으로 만들어졌습니다.


## 이미지 비교 분석

아래 예제는 여러개의 이미지를 비교해서 분석해주는 예제입니다.

In [9]:

def generate_image(file_path, prompt):

  img_data1 = get_obj_content(obj_loc="./liver1.png")
  image_obj1 = Part.from_data(data=base64.b64decode(img_data1), mime_type="image/png")

  img_data2 = get_obj_content(obj_loc="./liver2.png")
  image_obj2 = Part.from_data(data=base64.b64decode(img_data2), mime_type="image/png")

  model = GenerativeModel(model_name)

  responses = model.generate_content(
                                    [image_obj1, image_obj2, prompt],
                                    generation_config={
                                        "max_output_tokens": 2048,
                                        "temperature": 0.4,
                                        "top_p": 1,
                                        "top_k": 32
                                    },
                                    safety_settings=[],
                                    stream=False,
  )


  return responses.text

#------

prompt = """ 두개의 이미지를 각각 분석해주고, 두개의 이미지가 다른 점이 무엇인지 상세하게 설명해주세요.
"""

result = generate_image(file_path, prompt)
print(result)

 두 개의 이미지는 모두 복부 초음파 검사로 찍은 것입니다. 첫 번째 이미지는 정상적인 간을 보여줍니다. 간은 복부의 오른쪽 상단에 위치한 기관으로, 음식을 소화하고 신진대사를 조절하는 역할을 합니다. 정상적인 간은 매끈한 표면을 가지고 있으며, 혈류가 잘 통합니다.

두 번째 이미지는 간경화를 보여줍니다. 간경화는 간이 손상되어 흉터가 생기는 질환입니다. 간경화가 진행되면 간의 기능이 저하되고, 복수, 황달, 간암 등의 합병증이 발생할 수 있습니다. 간경화의 초음파 소견은 간의 표면이 불규칙해지고, 혈류가 감소하는 것입니다.

두 개의 이미지를 비교해보면, 정상적인 간은 매끈한 표면을 가지고 있고 혈류가 잘 통하는 반면, 간경화는 간의 표면이 불규칙해지고 혈류가 감소합니다. 이러한 차이는 간경화의 진단에 중요한 소견입니다.


## 수학 문제 해석

아래는 수학문제 풀이에 관한 신문 기사입니다. 테스트를 위해서 가져온 데이터 입니다. 다른 용도로 사용하지 말아주세요.
https://www.donga.com/news/Society/article/all/20090922/8812199/1


In [22]:


def generate_image(obj):
  model = GenerativeModel("gemini-pro-vision")
  responses = model.generate_content(
    [obj, """이미지 안에 있는 문제를 먼저 그대로 작성해주세요.
    해당 문제에 대해서 답을 구하는 절차를 최대한 상세하게 설명해주세요.
    계산결과 늘어난 길이는 얼마나 되나요 ?

     """],

    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings=[],
    stream=True,
    )

  for response in responses:
      print(response.text, end="")


img_data = get_obj_content(obj_loc='./math.png')
image1 = Part.from_data(data=base64.b64decode(img_data), mime_type="image/png")

generate_image(image1)

 그림과 같이 가로, 세로의 길이가 각각 5m, 4m인 직사각형 모양의 목장을 가로, 세로의 길이를 똑같은 길이만큼 늘렸더니 처음보다 넓이가 10m²만큼 늘었다. 이때 늘어난 길이를 구하여라.

문제를 먼저 그림으로 나타내보겠습니다.

[그림]

가로의 길이를 x라 하면, 세로의 길이는 4m이므로, 목장의 넓이는 5x(m)입니다.

늘어난 길이를 y라 하면, 늘어난 후의 가로의 길이는 (x+y)m, 세로의 길이는 (4+y)m가 됩니다.

늘어난 후의 목장의 넓이는 (x+y) * (4+y) (m)입니다.

늘어난 넓이가 10(m)이므로, (x+y) * (4+y) - 5x = 10

이 방정식을 풀면, y = 1(m)입니다.

따라서 늘어난 길이는 1m입니다.